## Core Functions
- Report the range of times for each event for the given data set (done)
- Report the median time for each for the selected data set (done)
- Report fastest for each event along with that person and the rest of their time (EX "Christina Hepper has the fastest Swim at _____ and had the following other times...., Jamie Fend had the fastest T1 at _____ and had teh following other times...." (done, but need to improve formatting of result)
- Create functions
-- function to convert to integer minutes (done)
-- function for performance standardizing (done)
- Compute the fastest theoretical time (done)
- Compute the median theoretical time
- Report how selected Bib Number did relative to lead (EX "989 was 0.9x at swimming, 1.1x at T1....")
- Create a line plot with each event on the X, the Standardized Time on the Y (standardize to fastest?) and each line being an athlete, highlight the user and the median line
- Need to cumsum the colums

## Future
- Use mongo db to store data
- Automatically scrape the results
- Compute a "competitiveness" score for each race (like FSAE world: https://fs-world.org/)
- Calculate the percentile for eahc event for the user

In [20]:
import pandas as pd
import numpy as np
#import pandas_bokeh
import plotly.express as px
import plotly.graph_objects as go
import hvplot.pandas
#pandas_bokeh.output_notebook()
from bokeh.plotting import figure, show
pd.set_option("precision", 2)

In [21]:
data = pd.read_excel('santa_cruz_data.xlsx', header = 0, index_col=None)

In [22]:
data.head()

,Bib,Name,Team Name,City,Start Time,Swim,T1,Bike,T2,Run,Chip Elapsed,Division,Div Place,Age,Age Place,Gender,Gender Place
0,1039,CHRISTINA HEPPER,NaN,MORGAN HILL,09:45:00,00:14:00,00:04:15,00:36:02,00:01:20,00:23:40,01:19:20,SPRINT-AGE GROUP,25.0,53.0,O/A,F,1.0
1,1006,JAMIE FEND,SANTA CRUZ TRIATHLON ASSOCIATION,APTOS,09:45:00,00:15:14,00:04:14,00:38:08,00:01:20,00:22:48,01:21:46,SPRINT-AGE GROUP,32.0,46.0,O/A,F,2.0
2,1112,ALEXANDRA LUZI,NaN,SAN FRANCISCO,09:45:00,00:12:50,00:06:41,00:38:04,00:01:48,00:22:44,01:22:08,SPRINT-AGE GROUP,35.0,30.0,O/A,F,3.0
3,1030,SUMI HACHMANN,TEEM SHEEPER,MENLO PARK,09:45:00,00:14:16,00:04:40,00:38:36,00:01:11,00:23:56,01:22:40,SPRINT-AGE GROUP,37.0,39.0,1,F,4.0
4,946,DENISE BUTLER,NaN,MOUNTAIN VIEW,09:45:00,00:12:34,00:04:37,00:40:08,00:00:49,00:24:46,01:22:56,SPRINT-AGE GROUP,39.0,34.0,1,F,5.0


We need to convert the DataTime objects contatined in the data frame to integer minutes to do all of our nice math. Use the function and create new columsn to hold the data.

Create a second set of parameters for cumulative times to use later

In [23]:
#create a function to convert the time to integer seconds
#To create a new column, use the [] brackets with the new column name at the left side of the assignment.

def convertTime (time):
    timeMinutes = (time.hour*60)+(time.minute)+(time.second/60)
    return timeMinutes

#convert to integers
data["Swim Minutes"] = data["Swim"].apply(convertTime)
data["T1 Minutes"] = data["T1"].apply(convertTime)
data["Bike Minutes"] = data["Bike"].apply(convertTime)
data["T2 Minutes"] = data["T2"].apply(convertTime)
data["Run Minutes"] = data["Run"].apply(convertTime)
#data["Elapsed Minutes"] = data["Chip Elapsed"].apply(convertTime)

#create cumulative times
data["Swim+T1"]=data["Swim Minutes"]+data["T1 Minutes"]
data["Plus Bike"]=data["Swim+T1"]+data["Bike Minutes"]
data["Plus T2"]=data["Plus Bike"]+data["T2 Minutes"]
data["Total"]=data["Plus T2"]+data["Run Minutes"]

Get statistical data about the event.

In [24]:
summary = data.describe()
data[["Swim Minutes", "T1 Minutes", "Bike Minutes", "T2 Minutes", "Run Minutes"]].describe()

,Swim Minutes,T1 Minutes,Bike Minutes,T2 Minutes,Run Minutes
count,26.00,26.00,26.00,26.00,26.00
mean,14.41,5.39,40.58,1.59,25.75
std,1.47,1.03,2.44,0.48,2.06
min,11.85,4.23,36.03,0.82,22.73
25%,13.52,4.62,39.20,1.29,24.12
50%,14.29,5.17,40.26,1.50,25.27
75%,15.15,5.75,41.93,1.86,27.25
max,18.30,8.07,47.55,2.97,30.08


In [25]:
#normalize the time for plotting
#select the data
dfPrescale = data[["Swim Minutes", "T1 Minutes", "Bike Minutes", "T2 Minutes", "Run Minutes"]]
normalized_df = ((dfPrescale - dfPrescale.min())/(dfPrescale.max()-dfPrescale.min()))*10

#rename the columns
normalized_df.rename(columns = {"Swim Minutes":"Swim Scaled", "T1 Minutes":"T1 Scaled", "Bike Minutes":"Bike Scaled", 
                                "T2 Minutes":"T2 Scaled", "Run Minutes":"Run Scaled"}, inplace = True)

#merge to the main frame
merged = pd.concat([data, normalized_df], axis=1)
merged['index1']=merged.index
merged.head()

,Bib,Name,Team Name,City,Start Time,Swim,T1,Bike,T2,Run,...,Swim+T1,Plus Bike,Plus T2,Total,Swim Scaled,T1 Scaled,Bike Scaled,T2 Scaled,Run Scaled,index1
0,1039,CHRISTINA HEPPER,NaN,MORGAN HILL,09:45:00,00:14:00,00:04:15,00:36:02,00:01:20,00:23:40,...,18.25,54.28,55.62,79.28,3.33,0.04,0.00,2.40,1.27,0
1,1006,JAMIE FEND,SANTA CRUZ TRIATHLON ASSOCIATION,APTOS,09:45:00,00:15:14,00:04:14,00:38:08,00:01:20,00:22:48,...,19.47,57.60,58.93,81.73,5.25,0.00,1.82,2.40,0.09,1
2,1112,ALEXANDRA LUZI,NaN,SAN FRANCISCO,09:45:00,00:12:50,00:06:41,00:38:04,00:01:48,00:22:44,...,19.52,57.58,59.38,82.12,1.52,6.39,1.77,4.57,0.00,2
3,1030,SUMI HACHMANN,TEEM SHEEPER,MENLO PARK,09:45:00,00:14:16,00:04:40,00:38:36,00:01:11,00:23:56,...,18.93,57.53,58.72,82.65,3.75,1.13,2.23,1.71,1.63,3
4,946,DENISE BUTLER,NaN,MOUNTAIN VIEW,09:45:00,00:12:34,00:04:37,00:40:08,00:00:49,00:24:46,...,17.18,57.32,58.13,82.90,1.11,1.00,3.56,0.00,2.77,4


In [26]:
#reduced = merged.drop(merged.columns[[2,3,4,5,6,7,8,9,10,11,13,14]], axis=1)
#reduced

In [27]:
# can send each runner data to a numnpy array that is named for that runner

columns = ['Swim Scaled', 'T1 Scaled', 'Bike Scaled', 'T2 Scaled', 'Run Scaled']

reduced.plot_bokeh(kind = 'scatter',
                  x='Name',
                  y=["Swim Minutes", "T1 Minutes"])


In [28]:
#assign user variable
user = "SAMANTHA DEWEES"
index = merged.index[merged['Name']==user].tolist()
merged.loc[index, ["Name", "Age", "Gender Place", "Swim Minutes", "T1 Minutes", "Bike Minutes", "T2 Minutes", "Run Minutes"]]

#data.loc[[user], ["Bib", "Name", "City", "Div Place", "Age"]]
#[data["Name"]=="SAMANTHA DEWEES"
#data.loc[data["Name"]=="SAMANTHA DEWEES"]

,Name,Age,Gender Place,Swim Minutes,T1 Minutes,Bike Minutes,T2 Minutes,Run Minutes
12,SAMANTHA DEWEES,25.0,13.0,15.03,7.07,39.65,2.22,23.63


In [29]:
def getMinTime (stage):
    minTime = data[stage].min()
    minIndex = data[stage].idxmin()
    #print("minTime is", minTime)
    #print("minIndex is", minIndex)
    return minTime, minIndex
    
def printResults (minIndex):
    #need to make this so it doesnt print reduntant colums or prints something nicely formatted
    print(data.iloc[minIndex,:])

In [30]:
events = ['Swim Minutes', 'T1 Minutes', 'Bike Minutes', 'T2 Minutes', 'Run Minutes']

shortestTime = 0
for stage in events:
    minTime, minIndex = getMinTime(stage)
    print("the fastest for ", stage)
    printResults(minIndex)
    shortestTime = shortestTime + minTime

print("the shortest possible time is: ", shortestTime)
#need to work on formatting

the fastest for  Swim Minutes
Bib                         1288
Name            CAROLE YOSHIWARA
Team Name                    NaN
City                   SUNNYVALE
Start Time              09:45:00
Swim                    00:11:51
T1                      00:05:18
Bike                    00:42:36
T2                      00:01:27
Run                     00:24:47
Chip Elapsed            01:26:02
Division        SPRINT-AGE GROUP
Div Place                   52.0
Age                         33.0
Age Place                      2
Gender                         F
Gender Place                10.0
Swim Minutes               11.85
T1 Minutes                   5.3
Bike Minutes                42.6
T2 Minutes                  1.45
Run Minutes                24.78
Swim+T1                    17.15
Plus Bike                  59.75
Plus T2                     61.2
Total                      85.98
Name: 9, dtype: object
the fastest for  T1 Minutes
Bib                                         1006
Name        

In [31]:
reduced2 = merged[["Name","Swim Minutes","Swim+T1","Plus Bike","Plus T2","Total","Gender Place"]]
#reduced2["Start"] = 0
reduced2 = reduced2.drop(25)
reduced2

color_nums = reduced2['Gender Place'].tolist()
color_names = reduced2['Name'].tolist()
colors_zip = zip(color_nums, color_names)

colors_zip

In [32]:
#hvplot.parallel_coordinates(reduced2[["Name","Swim Minutes","Swim+T1","Plus Bike","Plus T2","Total"]], "Name", width = 1280, height = 720)
type(reduced2.iat[0,0])

str

In [33]:
fig = px.parallel_coordinates(reduced2, color = "Gender Place", 
      labels={"Swim Minutes":"Time After Swim", "Swim+T1":"Time After First Transition", "Plus Bike":"Time After Bike", 
              "Plus T2":"Time After Second Transition", "Total":"Total Time"})

fig.show()

In [34]:
#reduced2=reduced2.head(3)
#reduced2

In [40]:
#fig = go.Figure(data=
 #   go.Parcoords(
  #      line = dict(color = reduced2['Gender Place'], colorscale = [[0.5,'lime'], [0.5, 'tomato'], [0.5, 'dodgerblue']]),
dimensions = list([
            dict(range = [11,18.5],
                label = 'Time After Swim', values = reduced2['Swim Minutes']),
            dict(range = [17,23.5],
                label = 'Time After First Transition', values = reduced2['Swim+T1']),
            dict(range = [54,67.5],
                label = 'Time After Bike', values = reduced2['Plus Bike']),
            dict(range = [55.5,69],
                label = 'Time After Second Transition', values = reduced2['Plus T2']),
            dict(range = [79,93.5],
                label = 'Total Time', values = reduced2['Total']),
            dict(range=[0,reduced2['Gender Place'].max()], tickvals = reduced2['Gender Place'], ticktext = reduced2['Name'],
                       label='Competitor', values=reduced2['Gender Place'])
        ])
 #   )
#)

#Parcords.line.colorscale = make_plotly(cl.scales['3']['qual']['Set2'], repeat=True)

fig = go.Figure(data=go.Parcoords(line = dict(color = reduced2['Gender Place'],
                   colorscale = [[0,'rgba(200,0,0,0.1)'],[0.5,'rgba(0,200,0,0.1)'],[1,'rgba(0,0,200,0.1)']]), dimensions=dimensions))

fig.update_layout(
    title="Triathalon Results"
)

fig.show()

https://stackoverflow.com/questions/64139316/plotly-how-to-insert-a-categorical-variable-into-a-parallel-coordinates-plot